In [7]:
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.models import load_model
from keras.preprocessing import image
import numpy as np
import cv2
from keras import backend as K
import tensorflow as tf


In [8]:
# Load your model
model = load_model('my_trained_model6.h5')

# Load the image with the desired target size
img_path = 'test.jpg'
img = image.load_img(img_path, target_size=(200, 200))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# Make a prediction
preds = model.predict(x)
class_idx = np.argmax(preds[0])
class_output = model.output[:, class_idx]

# Identify the name of the last convolutional layer in your model
# Adjust this according to your model architecture
last_conv_layer_name = 'conv2d_5'
last_conv_layer = model.get_layer(last_conv_layer_name)

# Use tf.GradientTape to compute gradients
with tf.GradientTape() as tape:
    tape.watch(last_conv_layer.output)  # Ensure that the layer's output is being watched
    preds = model(x)
    class_output = preds[:, class_idx]

# Compute gradients
grads = tape.gradient(class_output, last_conv_layer.output)[0]


# Average gradient per feature map
pooled_grads = K.mean(grads, axis=(0, 1, 2))

# Access the image and gradients to compute the Grad-CAM heatmap
iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])
pooled_grads_value, conv_layer_output_value = iterate([x])

# Multiply each channel in the feature map array by the gradient importance
for i in range(conv_layer_output_value.shape[-1]):
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

# Obtain the heatmap
heatmap = np.mean(conv_layer_output_value, axis=-1)
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)

# Use OpenCV to overlay the heatmap on the original image
img = cv2.imread(img_path)
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
superimposed_img = heatmap * 0.4 + img

cv2.imwrite('gradcam.jpg', superimposed_img)

1/1 [==============================] - 0s 216ms/step


RuntimeError: tf.gradients is not supported when eager execution is enabled. Use tf.GradientTape instead.